[pmhalvor/geo4460](https://github.com/pmhalvor/geo4460)   \
[pmhalvor@uio.no](mailto:pmhalvor@uio.no) \
GEO4460         

# Introduction


In this report, we will look into Digital Elevation Modeling (DEM). 
Specifically, we'll generate DEMs using the following techniques:

- Natural Neighbor Interpolation
- Triangulated Irregular Network (TIN)
- Topo to Raster ([Hutchinson's ANUDEM algorithm]( https://doi.org/10.1016/0022-1694(89)90073-5))

The goal of this exercise is to learn DEM generation techniques, understand how incorporating more informative data can improve DEM accuracy, and practice combining both quantitative and visual analysis methods.


# Data

The data used to build this analysis was extracted from the [N50 Dataset](https://kartkatalog.test.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac?search=n50), by Kartverket hosted on GeoNorge. 

We automated most of the workflow of this project in Python using the following software:

- [Whitebox-tools](https://www.whiteboxgeo.com/geospatial-software/): an open source Python library for geospatial analysis
- [Rasterio](https://rasterio.readthedocs.io/en/stable/index.html): a Python library for reading and writing geospatial raster data 
- [GRASS GIS](https://grass.osgeo.org/): an open source GIS software with a Python API and additional capabilities including GUI & Stream Burning
- [ArcGIS Pro](https://www.esri.com/en-us/arcgis/products/arcgis-pro/overview): for the ANUDEM algorithm and for visual quality assessments

The code for this project is found at [github.com/pmhalvor/geo4460](https:/github.com/pmhalvor/geo4460).

## Preprocessing

Before we could go through the steps outlined in the project description, we needed to convert the raw N50 data to a similar geodatabase as our example data, containing feature layers `contour_arc`, `elevation_point`, `lake_polygon`, and `river_arc`.

To do so we used ArcGIS Pro to run the following steps:

1. Import the N50 data into a new geodatabase.
1. Load the land cover data, elevation points, and contour lines to the current map.
1. Create a feature layer containing a single polygon (rectangle) around the area of interest. We chose a region in Rogaland around Preikestolen. 
1. Clip the N50 layers to the rectangle polygon.
1. Convert labels in the land cover data to new feature layer polygons when the object type was `innsjø`, `innsjø regulert`, or `elv`.
1. Convert river feature layers to polylines.
1. Rename relevant layers to `contour_arc`, `elevation_point`, `lake_polygon`, and `river_arc`.
1. Export the layers to a new geodatabase.

These steps were executed in ArcGIS Pro due to limitations of open-source tools for reading, parsing, and storing `.gdb` files.
Also, performing these steps manually in a GUI ensured the selection of an interesting region to analyze within the boundaries of our data, providing a better foundation for understanding the results later. 



Two additional preprocessing steps were performed directly in our Python workflow. 
These final preprocessing steps enforced the correct coordinate system across our layers and converted our contour lines to points, for more informative DEM interpolation. 
Generating points from contours dramatically improves the quality of simpler DEMs, as will be discussed later.


# Methods

## DEM generation

For this specific analysis, we explored 3 interpolation methods: Natural Neighbor, TIN Gridding, and ANUDEM (Topo to Raster in ArcGIS Pro).
The first two methods were automated and implemented in Python using Whitebox-tools and Rasterio; the last method was manually implemented in ArcGIS Pro.

First, one DEM was generated for each of the automated methods using only the contour points. 
Then, another DEM using only the _elevation_ points was generated for each of the automated methods.
This was deemed possible after recognizing the expected inputs for the two methods required only a point layer. 

For the more complex ANUDEM method, we provided all the information included in the example data: contours, elevation points, lakes, and rivers (streams).
The raster output from this method extended further than the intended area of interest, so a mask was applied to extract only the region we were investigating.
During this extraction, we made sure to store the output to the same directory where our other data was, in order to include this ANUDEM result in the quality assessment later. 

An extra DEM was created as an attempt to approximate the ANUDEM method using the TIN method.
This was achieved by merging the TIN raster with a stream-burned raster created from the river feature layer.
The resulting layer yielded results comparable to the TIN gridded DEM, although still less accurate than the ANUDEM method.
We only generated this DEM using the TIN grid DEM generated from contour points. 


## Quality assessment
The quality assessment of the DEMs was done using a combination of visual and quantitative analysis.

### RMSE
Quantitative analysis involved calculating the Root Mean Square Error (RMSE) between each generated DEM and the original elevation points.
The RMSE was calculated using the following formula:
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^{n}(z_i - z_{pred})^2}
$$
where $(z_i)$ is the original elevation point value, $(z_{pred})$ is the predicted elevation from the DEM at that point's location, and $(n)$ is the number of points.
RMSE was calculated for all generated DEMs by comparing their predicted elevations to the known values at the original elevation point locations.
A similar RMSE analysis using the contour points could have been performed but was omitted due to time constraints.


### Hillshade
The first visual analysis involved generating hillshade rasters from each of the DEMs.
Hillshade is a technique used to create a shaded relief map, which can help visualize the terrain and identify features such as ridges, valleys, and slopes.
Only the DEM was needed as input, with optional parameters the azimuth and altitude for the light source.
In Whitebox-tools, default values are set to 315° and 30°, which were used for our analysis. 
Hillshade rasters were generated for each DEM in our automated workflow, and the results were visually compared in ArcGIS Pro.

### Slope
Similar to hillshade, slope is another means of visualizing the terrain.
Again, only the DEM is needed as input, though units and zfactor could be set if needed. 
Simlar to before, slope rasters were generated in our workflow, and visually analyzed in ArcGIS Pro.


### Profile analysis
Further visual analysis involved plotting elevation profiles along an arbitrary transect line. 
Whitebox-tools was used to generate the profile analysis, storing the output for each DEM in an HTML file. 
To combine all profiles into a single Matplotlib plot, Beautiful Soup was used to parse these HTML files and extract the necessary data into arrays.

A transect line was created using the `create_line` function in Whitebox-tools, which creates a line between two points.
The start and stop points for the transect were automatically selected from the extent of the input data. 
An inset was also defined, to ensure the transect line is not too close to the edges of the DEM, which could result in some artifacts in the profile.


# Results

## RMSE results
| DEM Type                        | RMSE (m)          | N Points |
|---------------------------------|-------------------|----------|
| Natural Neighbor (Contour)      | 17.45             | 1212     |
| Natural Neighbor (Points)       | 2867.71           | 1212     |
| TIN Gridding (Contour)          | 17.47             | 1212     |
| TIN Gridding (Points)           | 2867.71           | 1212     |
| Stream Burn (Contour TIN based) | 17.48             | 1212     |
| ANUDEM (ArcGIS Pro)             | 15.00             | 1212     |

**Table 1**: RMSE results calculated against the original elevation points for each generated DEM. Values rounded to two decimal places.


## Hillshade results
![Hillshade results](img/hillshade.png) 

**Figure 1**: Hillshade results for each generated DEM.

## Slope results
![Slope results](img/slope.png)

**Figure 2**: Slope results for each generated DEM.

## Profile results
![Profile results](img/combined_profiles_plot.png)

**Figure 3**: Elevation profiles along the transect line for each generated DEM.

# Discussion 

The results from the RMSE analysis displayed in Table 1 show that the ANUDEM method performed best, with an RMSE of 15.00 m when evaluated against the original elevation points.
However, when considering the profile analysis (Figure 3), it became apparent why this method outperformed the automated methods.
Note the elevations _below zero_ in Figure 3. 
Only the ANUDEM method estimated terrain elevations below sea level, resulting in a much more realistic DEM for this fjord.
The other methods, Natural Neighbor and TIN Gridding, both estimated the terrain to be entirely above sea level, flattening out at zero elevation, since no contour data exists below sea level in our input dataset.
Excluding the influence of these below-sea-level areas, our automated methods built from contour points seemed to match the terrain almost identically to the ANUDEM method, with RMSE values around 17.45 m.

As mentioned, we also wanted to compare DEMs generated solely from the sparse elevation points instead of the denser contour-derived points.
The resulting RMSE values for those DEMs were significantly higher, around 2867.71 m.
This large discrepancy is likely because the elevation points are not evenly distributed across the terrain and thus did not provide a good representation of the true surface when used alone for interpolation.

## Hillshade and slope 
Hillshade and slope results were presented in Figures 1 and 2, respectively.
These visualizations further confirm that the recommended approach for generating DEMs in this context is to use contour points as the primary input.

The middle columns of both figures show the results from DEMs generated using only elevation points, which appear blurred, very coarse, and not very informative.
This was expected, since minimal preprocessing was done on the elevation points, and their spatial distribution was uneven.

The stream-burning approach (applied to the TIN contour DEM) seems to yield the most fine-grained appearance in both figures. 
However, the standard automated methods (Natural Neighbor and TIN from contours) and ANUDEM also appear detailed and informative.

Upon closer inspection, it becomes apparent that the ANUDEM method produced the fewest visual artifacts, particularly in the fjord areas.
Both of the simpler automated methods exhibited some obvious artifacts across the fjord, which were not present in the ANUDEM result.
It should also be mentioned that judging by the slope results alone (Figure 2), the Natural Neighbor method (from contours) had relatively few artifacts and appeared comparable to the ANUDEM method.


# Conclusion

In this report, we have explored different methods for generating DEMs using contour lines and elevation points.
We have also performed a quality assessment of the generated DEMs using RMSE, hillshade, slope, and profile analysis.

The process was automated as much as possible using Python libraries to facilitate reproducibility and application to new datasets. 
However, limitations to the automation were the need to use ArcGIS Pro for the ANUDEM method and the initial manual preparation of the data as a geodatabase.
Everything else was possible to implement in Python, generating results comparable to the ANUDEM method, especially when using contour-derived points.

For the best overall results in this specific case, we recommend using the ANUDEM method, as it produced the most realistic DEM, effectively modeling elevations below sea level which were absent in the contour data. 
However, due to its proprietary nature, the ANUDEM method is not as accessible as the other methods.
Simpler open-source methods like TIN Gridding and Natural Neighbor interpolation remain viable options, particularly when working with dense input data like contour-derived points. 

When possible, we recommend using points derived from contour lines rather than sparse, stand-alone elevation points to generate DEMs.
Contour layers typically provide more comprehensive spatial coverage, assuming they originate from a reliable source. 
In our case, using the N50 data from Kartverket, with a [FAIR status of 0.95](https://register.test.geonorge.no/fair-register/n50-kartdata/5afec3ea-2ba0-402f-93d5-a7fcbd383284#fair), we can be confident that the contour lines are a good representation of the terrain in this region. 

# References


- [John Lindsay](https://github.com/jblindsay) et. al, _Whitebox tools_, [https://github.com/jblindsay/whitebox-tools](https://github.com/jblindsay/whitebox-tools)
- [Sean Gillies](https://github.com/sgillies) et. al, _Rasterio_, [https://github.com/rasterio/rasterio](https://github.com/rasterio/rasterio)
- GRASS Development Team, _GRASS GIS_, [https://grass.osgeo.org](https://grass.osgeo.org/)
- ESRI, _ArcGIS Pro_, [https://www.esri.com/en-us/arcgis/products/arcgis-pro/overview](https://www.esri.com/en-us/arcgis/products/arcgis-pro/overview)
- Kartverket, _N50 Kartdata_, [https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac?search=n50](https://kartkatalog.geonorge.no/metadata/n50-kartdata/ea192681-d039-42ec-b1bc-f3ce04c189ac?search=n50)
